In [0]:
%python
df_data = (
    spark.read
         .option("recursiveFileLookup", "true")
         .option("multiLine", "true")
         .json("/Volumes/workspace/default/json_raw_volume/IOT Device Status Messages/")
)

df_data.printSchema() 

root
 |-- Body: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- mnsn: string (nullable = true)
 |    |-- scheduleName: string (nullable = true)
 |    |-- smsProperties: struct (nullable = true)
 |    |    |-- device: struct (nullable = true)
 |    |    |    |-- address: string (nullable = true)
 |    |    |    |-- disposalMarkerList: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- amount: struct (nullable = true)
 |    |    |    |    |    |    |-- capacity: string (nullable = true)
 |    |    |    |    |    |    |-- state: string (nullable = true)
 |    |    |    |    |    |    |-- typical: string (nullable = true)
 |    |    |    |    |    |    |-- unit: string (nullable = true)
 |    |    |    |    |    |-- color: string (nullable = true)
 |    |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- fam

In [0]:
from pyspark.sql.functions import col, lit, struct, array

df_transformed_full = df_data.select(
    col("Body.mnsn").alias("_id"),
    col("Body.timestamp").cast("long").alias("createTimestamp"),
    col("Body.type").alias("type"),
    lit("assigned").alias("attribute"),
    col("Properties.relatedGroupId").alias("relatedGroupId"),
    lit(None).cast("array<string>").alias("tagIds"),
    lit(None).cast("string").alias("communicationStatus"),
    col("Properties.customerId").alias("customerId"),
    col("Properties.dealerId").alias("dealerId"),
    lit(None).cast("string").alias("L1DealerId"),
    lit(None).cast("string").alias("L2DealerId"),
    lit(None).cast("string").alias("L3DealerId"),
    lit(None).cast("string").alias("divisionId"),
    lit(None).cast("boolean").alias("onContract"),
    lit(None).cast("long").alias("purchaseDate"),
    lit(None).cast("string").alias("equipmentId"),
    lit(None).cast("boolean").alias("isSNAP"),
    lit(None).cast("string").alias("servicingDealerId"),
    lit(None).cast("string").alias("SAP_name"),
    lit(None).cast("string").alias("SAP_location"),

    # deviceGeneral full
    struct(
        lit(None).cast("string").alias("relatedAgentId"),
        col("Body.smsProperties.device.modelName").alias("modelName"),
        col("Body.smsProperties.device.serialId").alias("serialNumber"),
        col("Body.smsProperties.device.familyName").alias("familyName"),
        col("Body.smsProperties.device.friendlyName").alias("friendlyName"),
        lit(None).cast("int").alias("dispErrorCode"),
        lit(None).cast("string").alias("location"),
        lit(None).cast("int").alias("statusCode"),
        lit(None).cast("array<int>").alias("status"),
        lit(None).cast("int").alias("troubleCode"),
        lit(None).cast("boolean").alias("dskFlag"),
        lit(None).cast("string").alias("firmwareVersion"),
        lit(None).cast("string").alias("subnetMask"),
        lit(None).cast("string").alias("gateway"),
        lit(None).cast("long").alias("lastStatusUpdate"),
        lit(None).cast("long").alias("lastSuppliesUpdate"),
        lit(None).cast("long").alias("lastCounterUpdate"),
        lit(None).cast("long").alias("lastBasicUpdate"),
        lit(None).cast("long").alias("lifeCount"),
        lit(None).cast("string").alias("detectedDeviceType"),
        lit(None).cast("string").alias("productFamilyName"),
        # forecastingList
        lit(None).cast("array<struct<color:string,printCount:int,tonerEndDate:timestamp,tonerNumber:int,tonerRemaining:int>>").alias("forecastingList")
    ).alias("deviceGeneral"),

    # printerDevice simplified
    struct(
        struct(
            struct(
                lit(None).cast("string").alias("value"),
                lit(None).cast("long").alias("ipToLong")
            ).alias("ipAddress"),
            struct(
                lit(None).cast("int").alias("http"),
                lit(None).cast("int").alias("https")
            ).alias("port"),
            struct(
                lit(None).cast("string").alias("snmpVersion"),
                lit(None).cast("int").alias("retry"),
                lit(None).cast("int").alias("timeout"),
                lit(None).cast("string").alias("getCommunityString"),
                lit(None).cast("string").alias("contextName"),
                lit(None).cast("string").alias("userName"),
                lit(None).cast("string").alias("securityAuthType"),
                lit(None).cast("string").alias("securityAuthKey"),
                lit(None).cast("string").alias("securityPrivType"),
                lit(None).cast("string").alias("securityPrivKey")
            ).alias("snmpSetting")
        ).alias("connectionSetting"),
        lit(None).cast("string").alias("macAddress"),
        lit(None).cast("int").alias("statusCode")
    ).alias("printerDevice"),

    # interface & lists
    col("Body.smsProperties.interface").alias("interface"),
    col("Body.smsProperties.device.inTrayList").alias("inTrayList"),
    col("Body.smsProperties.device.outTrayList").alias("outTrayList"),
    col("Body.smsProperties.device.disposalMarkerList").alias("disposalMarkerList"),
    col("Body.smsProperties.device.markerList").alias("markerList"),

    lit(None).cast("array<struct<id:string,value:string>>").alias("maintenanceCodeList"),
    lit(None).cast("array<struct<id:string,value:string>>").alias("serviceCodeList")
)
df_transformed_full.printSchema()

root
 |-- _id: string (nullable = true)
 |-- createTimestamp: long (nullable = true)
 |-- type: string (nullable = true)
 |-- attribute: string (nullable = false)
 |-- relatedGroupId: string (nullable = true)
 |-- tagIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- communicationStatus: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- dealerId: string (nullable = true)
 |-- L1DealerId: string (nullable = true)
 |-- L2DealerId: string (nullable = true)
 |-- L3DealerId: string (nullable = true)
 |-- divisionId: string (nullable = true)
 |-- onContract: boolean (nullable = true)
 |-- purchaseDate: long (nullable = true)
 |-- equipmentId: string (nullable = true)
 |-- isSNAP: boolean (nullable = true)
 |-- servicingDealerId: string (nullable = true)
 |-- SAP_name: string (nullable = true)
 |-- SAP_location: string (nullable = true)
 |-- deviceGeneral: struct (nullable = false)
 |    |-- relatedAgentId: string (nullable = true)
 |    |-

In [0]:
df_transformed_full.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("df_transformed")

In [0]:
%sql
SELECT * FROM df_transformed;


_id,createTimestamp,type,attribute,relatedGroupId,tagIds,communicationStatus,customerId,dealerId,L1DealerId,L2DealerId,L3DealerId,divisionId,onContract,purchaseDate,equipmentId,isSNAP,servicingDealerId,SAP_name,SAP_location,deviceGeneral,printerDevice,interface,inTrayList,outTrayList,disposalMarkerList,markerList,maintenanceCodeList,serviceCodeList
mn=TVg1MDcwTg==:sn=STU0NjQz,1765670556160,offline,assigned,2344cb97-4ebe-4412-8a8c-64730003681c,null,null,ea387dfe-357c-48b7-818a-e01063c184f5,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null)","List(List(List(null, null), List(null, null), List(null, null, null, null, null, null, null, null, null, null)), null, null)",null,null,null,null,null,null,null
mn=QlA1MEMyNg==:sn=MjUxMjA1NTAwMA==,1765679285143,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP BP-50C26, 2512055000, SHARP-155, , null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null)","List(List(List(null, null), List(null, null), List(null, null, null, null, null, null, null, null, null, null)), null, null)","List(List(List(14-50-51-CA-30-99, 1, 6), List(, 3, 24)), List(List(127.0.0.1, 172.29.240.22, 3, 255.0.0.0), List(172.29.241.167, 172.29.240.22, 1, 255.255.254.0)))","List(List(List(100, empty, 0, sheets), Bypass Tray, 1, false, true, -2, -2, unknown, other, -2, unknown, micrometers, -2, custom-media-type-stationery-stationery-1, Bypass Tray, Bypass Tray, false), List(List(550, , 181, sheets), Tray 1, 2, false, false, 210000, 297000, iso-a4-white, 210000 x 297000, 297000, A4, micrometers, 210000, custom-media-type-stationery-stationery-1, TRAY 1, Tray 1, false), List(List(550, empty, 0, sheets), Tray 2, 3, false, false, 420000, 297000, iso-a3-white, 420000 x 297000, 297000, A3R, micrometers, 420000, custom-media-type-stationery-stationery-1, BP-DE12, Tray 2, false), List(List(, unknown, , sheets), Auto Select, 31, false, false, -2, -2, unknown, other, -2, unknown, micrometers, -2, unknown, Auto Select, Auto Select, false))","List(List(List(400, notFull, , sheets), faceDown, Center Tray, 1, Center Tray, Center Tray, firstToLast))","List(List(List(, dis_ok, 100, percent), unknown, Waste Toner, wasteToner))","List(List(List(100, ok, 14, percent), cyan, Cyan Toner, toner), List(List(100, ok, 26, percent), magenta, Magenta Toner, toner), List(List(100, ok, 94, percent), yellow, Yellow Toner, toner), List(List(100, ok, 5, percent), black, Black Toner, toner), List(List(100, ok, 95, percent), cyan, Cyan Photoconductive Drum (DC), opc), List(List(100, ok, 95, percent), magenta, Magenta Photoconductive Drum (DM), opc), List(List(100, ok, 95, percent), yellow, Yellow Photoconductive Drum (DY), opc), List(List(100, ok, 93, percent), black, Black Photoconductive Drum (DK), opc))",null,null
mn=TVgzNTYwTg==:sn=NzUwNjkzMjYwMA==,1765679285318,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP MX-3560N, 7506932600, SHARP-112, , null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null)","List(List(List(null, null), List(null, null), List(null, null, null, null, null, null, null, null, null, null)), null, null)","List(List(List(80-38-96-69-12-A9, 1, 6), List(, 2, 24)), List(List(127.0.0.1, 172.29.242.129, 2, 255.0.0.0), List(172.29.242.156, 172.29.242.129, 1, 255.255.255.128)))","List(List(List(100, empty, 0, sheets), Bypass Tray, 1, false, true, -2, -2, unknown, other, -2, unknown, micrometers, -2, stationery, Bypass Tray, Bypass Tray, false), List(List(600, , 198, sheets), Tray 1, 2, false, false, 210000, 297000, iso-a4-white, 210000 x 297000, 

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc

window_spec = Window.partitionBy("_id").orderBy(desc("createTimestamp"))

df_dedup = (
    df_transformed_full
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)


In [0]:
df_dedup.display()

_id,createTimestamp,type,attribute,relatedGroupId,tagIds,communicationStatus,customerId,dealerId,L1DealerId,L2DealerId,L3DealerId,divisionId,onContract,purchaseDate,equipmentId,isSNAP,servicingDealerId,SAP_name,SAP_location,deviceGeneral,printerDevice,interface,inTrayList,outTrayList,disposalMarkerList,markerList,maintenanceCodeList,serviceCodeList
mn=QlA0MUMzNg==:sn=NDQxMjgwMjI1NQ==,1765765685979,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP BP-41C36, 4412802255, SHARP-193, , null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null)","List(List(List(null, null), List(null, null), List(null, null, null, null, null, null, null, null, null, null)), null, null)","List(List(List(8C-52-19-D4-F5-0E, 1, 6), List(84-25-3F-B8-58-D0, 2, 71), List(, 3, 24)), List(List(10.36.102.50, 10.36.102.1, 1, 255.255.255.0), List(127.0.0.1, 10.36.102.1, 3, 255.0.0.0)))","List(List(List(120, empty, 0, sheets), Bypass Tray, 1, false, true, -2, -2, unknown, other, -2, unknown, micrometers, -2, custom-media-type-stationery-stationery-1, Bypass Tray, Bypass Tray, false), List(List(600, , 198, sheets), Tray 1, 2, false, false, 420000, 297000, iso-a3-white, 420000 x 297000, 297000, A3R, micrometers, 420000, custom-media-type-stationery-stationery-1, TRAY 1, Tray 1, false), List(List(600, , 198, sheets), Tray 2, 3, false, false, 364000, 257000, jis-b4-white, 364000 x 257000, 257000, B4R, micrometers, 364000, custom-media-type-stationery-stationery-1, BP-DE15-1, Tray 2, false), List(List(1350, , 445, sheets), Tray 3, 4, false, false, 210000, 297000, iso-a4-white, 210000 x 297000, 297000, A4, micrometers, 210000, custom-media-type-stationery-stationery-1, BP-DE15-2, Tray 3, false), List(List(950, , 475, sheets), Tray 4, 5, false, false, 182000, 257000, jis-b5-white, 182000 x 257000, 257000, B5, micrometers, 182000, custom-media-type-stationery-stationery-1, BP-DE15-3, Tray 4, false), List(List(, unknown, , sheets), Auto Select, 31, false, false, -2, -2, unknown, other, -2, unknown, micrometers, -2, unknown, Auto Select, Auto Select, false))","List(List(List(400, notFull, , sheets), faceDown, Center Tray, 1, Center Tray, Center Tray, firstToLast), List(List(120, , 120, sheets), faceDown, Upper Tray, 3, MX-TR20, Upper Tray, firstToLast))","List(List(List(, failure, , percent), unknown, Waste Toner, wasteToner))","List(List(List(100, ok, 29, percent), cyan, Cyan Toner, toner), List(List(100, ok, 31, percent), magenta, Magenta Toner, toner), List(List(100, ok, 24, percent), yellow, Yellow Toner, toner), List(List(100, ok, 1, percent), black, Black Toner, toner), List(List(100, ok, 99, percent), cyan, Cyan Photoconductive Drum (DC), opc), List(List(100, ok, 99, percent), magenta, Magenta Photoconductive Drum (DM), opc), List(List(100, ok, 99, percent), yellow, Yellow Photoconductive Drum (DY), opc), List(List(100, ok, 99, percent), black, Black Photoconductive Drum (DK), opc))",null,null
mn=QlA1MEMyNg==:sn=MjUxMjA1NTAwMA==,1765765685118,smsProps,assigned,6de069dd-89d2-43b5-bc64-15ffbba5ccd6,null,null,b8bf935d-026c-426a-b9f2-ef97d01bf973,281083,null,null,null,null,null,null,null,null,null,null,null,"List(null, SHARP BP-50C26, 2512055000, SHARP-155, , null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null)","List(List(List(null, null), List(null, null), List(null, null, null, null, null, null, null, null, null, null)), null, null)","List(List(List(14-50-51-CA-30-99, 1, 6), List(, 3, 24)), List(List(127.0.0.1, 172.29.240.22, 3, 255.0.0.0), List(172.29.241.167, 172.29.240.22, 1, 255.255.254.0)))","List(List(List(100, empty, 0, sheets), Bypass Tray, 1, false, true, -2, -2, unknown, other, -2, unknown, micrometers, -2, custom-media-type-stationery-stationery-1, Bypass Tray, Bypass Tray, false), List(List(550, , 181, sheets), Tray 1, 2, false, false

In [0]:
all_columns = df_dedup.columns
print(all_columns)


['_id', 'createTimestamp', 'type', 'attribute', 'relatedGroupId', 'tagIds', 'communicationStatus', 'customerId', 'dealerId', 'L1DealerId', 'L2DealerId', 'L3DealerId', 'divisionId', 'onContract', 'purchaseDate', 'equipmentId', 'isSNAP', 'servicingDealerId', 'SAP_name', 'SAP_location', 'deviceGeneral', 'printerDevice', 'interface', 'inTrayList', 'outTrayList', 'disposalMarkerList', 'markerList', 'maintenanceCodeList', 'serviceCodeList']


In [0]:
# Dynamic column mapping for whenMatchedUpdate
update_dict = {col_name: f"src.{col_name}" for col_name in all_columns if col_name != "_id"}


In [0]:
from delta.tables import DeltaTable

# Load the target Delta table
delta_table = DeltaTable.forName(spark, "df_transformed")

# Merge source into target
delta_table.alias("tgt").merge(
    df_dedup.alias("src"),
    "tgt._id = src._id"  # Matching condition
).whenMatchedUpdate(set=update_dict)\
 .whenNotMatchedInsertAll()\
 .execute()


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]